### Loading Libraries & Environment Variables

In [ ]:
import os
import requests
import json
import prettytable
import pandas as pd
from dotenv import load_dotenv

In [10]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

### Loading Data

#### Pulling labor market data with the BLS API

In [11]:
# Set the base url for BLS API v2
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'

In [12]:
# Pull the API key in .env which contains: bls_key = 'key'
bls_key = os.getenv("bls_key")
key = '?registrationkey={}'.format(bls_key)

In [13]:
# Set the start year and end year for the data series
dates = ('2012', '2022')

In [14]:
# Create a dictionary of the series codes and names.
series_dict = {
    'LAUCN010010000000005': 'Employment',
    'LAUCN010010000000006': 'Labor_Force'}

In [19]:
# The following code is modeled from an example provided at https://www.bd-econ.com/blsapi.html.

# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series and the dates as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

# Post request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']


In [20]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Create an empty dataframe to fill with values
df_labor_market = pd.DataFrame()

# Build a pandas series from the API results, p
for s in p:
    df_labor_market[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        data = [i['value'] for i in s['data']]
        ).astype(float).iloc[::-1]
# Review last 5 results
df_labor_market.tail()


,Employment,Labor_Force
2021-07-01,24988.0,25710.0
2021-08-01,24785.0,25517.0
2021-09-01,24685.0,25326.0
2021-10-01,24759.0,25337.0
2021-11-01,25069.0,25722.0


In [21]:
# Review
df_labor_market.head()

,Employment,Labor_Force
2012-01-01,23452.0,25462.0
2012-02-01,23546.0,25507.0
2012-03-01,23598.0,25457.0
2012-04-01,23800.0,25484.0
2012-05-01,23976.0,25733.0


In [22]:
# Create the fips code series to append to the dataframe
counties=len(df_labor_market)
seriesid  = list(series_dict.keys())
fips_code = seriesid[0]
fips_code = fips_code[6:10]

fips_list = []
for i in range(1,counties+1):
    fips_list.append(fips_code)

In [24]:
df_labor_market['Fips_Code'] = fips_list

In [27]:
#Review the dataframe again
df_labor_market.head()

,Employment,Labor_Force,Fips_Code
2012-01-01,23452.0,25462.0,1001
2012-02-01,23546.0,25507.0,1001
2012-03-01,23598.0,25457.0,1001
2012-04-01,23800.0,25484.0,1001
2012-05-01,23976.0,25733.0,1001
